In [5]:
# install.packages("lme4", repos = "http://cran.us.r-project.org")
# install.packages("dplyr", repos = "http://cran.us.r-project.org")
# install.packages("tibble" , repos = "http://cran.us.r-project.org")
# install.packages("ggplot2", repos = "http://cran.us.r-project.org")

package 'tibble' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\shilongz\AppData\Local\Temp\RtmpcPNSY6\downloaded_packages
package 'ggplot2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\shilongz\AppData\Local\Temp\RtmpcPNSY6\downloaded_packages


In [6]:
library(lme4)
library(dplyr)

Loading required package: Matrix

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [7]:
getwd()

[1] "D:/workspace/Github/machine-learning-use-cases/customer-ltv"

In [12]:
data <- read.csv('ltv.csv')

In [13]:
head(data)

CUSTOMER_ID,PRODUCT_TYPE,CHURN_STATUS,DURATION,TRAFFIC_SOURCE,GENDER,AGE_BUCKET,STATE,CARRIER_NAME,SUBMIT_METHOD,REVENUE_RECEIVED
C1,Product B,1,365,Source 43,M,Age Range 1,FL,Carrier 5,Web,52.00
C2,Product B,0,1072,Source 95,F,Age Range 1,NC,Carrier 3,Phone,228.00
C3,Product B,1,1091,Source 173,M,Age Range 1,MN,Carrier 3,Web,183.00
C4,Product A,0,405,Source 2,F,Age Range 1,GA,Carrier 3,Phone,586.31
C5,Product A,1,360,Source 215,M,Age Range 1,TX,Carrier 5,Phone,411.00
C6,Product A,0,229,Source 160,M,Age Range 1,ME,Carrier 1,Web,606.00


In [31]:
data %>%
  select(CUSTOMER_ID, REVENUE_RECEIVED, TRAFFIC_SOURCE,GENDER,AGE_BUCKET,STATE,CARRIER_NAME,SUBMIT_METHOD) -> data.copy

In [32]:
head(data.copy)

CUSTOMER_ID,REVENUE_RECEIVED,TRAFFIC_SOURCE,GENDER,AGE_BUCKET,STATE,CARRIER_NAME,SUBMIT_METHOD
C1,52.00,Source 43,M,Age Range 1,FL,Carrier 5,Web
C2,228.00,Source 95,F,Age Range 1,NC,Carrier 3,Phone
C3,183.00,Source 173,M,Age Range 1,MN,Carrier 3,Web
C4,586.31,Source 2,F,Age Range 1,GA,Carrier 3,Phone
C5,411.00,Source 215,M,Age Range 1,TX,Carrier 5,Phone
C6,606.00,Source 160,M,Age Range 1,ME,Carrier 1,Web


In [17]:
summary(data)

  CUSTOMER_ID       PRODUCT_TYPE    CHURN_STATUS       DURATION     
 C1     :    1   Product A:26728   Min.   :0.0000   Min.   :   0.0  
 C10    :    1   Product B:10831   1st Qu.:0.0000   1st Qu.: 225.0  
 C100   :    1   Product C: 2441   Median :0.0000   Median : 445.0  
 C1000  :    1                     Mean   :0.4106   Mean   : 599.8  
 C10000 :    1                     3rd Qu.:1.0000   3rd Qu.: 895.0  
 C10001 :    1                     Max.   :1.0000   Max.   :4780.0  
 (Other):39994                                                      
    TRAFFIC_SOURCE  GENDER          AGE_BUCKET        STATE      
 Source 1  : 8936   F:20936   Age Range 1:22659   TX     : 3357  
 Source 95 : 6337   M:19064   Age Range 2:12475   FL     : 3322  
 Source 178: 2011             Age Range 3: 4866   CA     : 2386  
 Source 222: 1802                                 NY     : 1794  
 Source 160: 1367                                 GA     : 1789  
 Source 164: 1054                                 OH

In [19]:
data %>%
  filter(PRODUCT_TYPE == 'Product A') %>%  # only predict product A
  filter(CHURN_STATUS == 1) %>%  # only focus on churn customers
  select(TRAFFIC_SOURCE,GENDER,AGE_BUCKET,STATE,CARRIER_NAME,SUBMIT_METHOD, REVENUE_RECEIVED) %>%
  mutate(traffic_source = factor(TRAFFIC_SOURCE)) %>%
  mutate(gender = factor(GENDER)) %>%
  mutate(age_bucket = factor(AGE_BUCKET)) %>%
  mutate(state = factor(STATE)) %>%
  mutate(carrier = factor(CARRIER_NAME)) %>%
  mutate(submit_method= factor(SUBMIT_METHOD)) %>%
  mutate(revenue = REVENUE_RECEIVED) %>%
  select(traffic_source, gender, age_bucket, state, carrier, submit_method, revenue) -> ltv

In [20]:
summary(ltv)

    traffic_source gender         age_bucket       state           carrier    
 Source 1  :3246   F:6416   Age Range 1:5849   FL     :1294   Carrier 1:2107  
 Source 95 :2284   M:5376   Age Range 2:4929   TX     :1082   Carrier 2: 676  
 Source 178: 577            Age Range 3:1014   NY     : 681   Carrier 3:5768  
 Source 222: 501                               OH     : 591   Carrier 4:1228  
 Source 164: 376                               CA     : 587   Carrier 5:2013  
 Source 2  : 283                               GA     : 574                   
 (Other)   :4525                               (Other):6983                   
 submit_method    revenue      
 Phone:11168   Min.   :   0.0  
 Web  :  624   1st Qu.: 235.1  
               Median : 478.9  
               Mean   : 516.3  
               3rd Qu.: 713.0  
               Max.   :2374.3  
                               

In [21]:
ltv.model <- lmer(revenue ~ gender + age_bucket + state + carrier + submit_method + (1|traffic_source), data=ltv)

In [22]:
summary(ltv.model)


Correlation matrix not shown by default, as p = 58 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Linear mixed model fit by REML ['lmerMod']
Formula: revenue ~ gender + age_bucket + state + carrier + submit_method +  
    (1 | traffic_source)
   Data: ltv

REML criterion at convergence: 169958.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4702 -0.7540 -0.0948  0.6451  5.3304 

Random effects:
 Groups         Name        Variance Std.Dev.
 traffic_source (Intercept)  20935   144.7   
 Residual                   109454   330.8   
Number of obs: 11792, groups:  traffic_source, 164

Fixed effects:
                      Estimate Std. Error t value
(Intercept)            433.110     26.292  16.473
genderM                 21.376      6.177   3.461
age_bucketAge Range 2   25.504      6.625   3.850
age_bucketAge Range 3   89.878     11.398   7.886
stateAR                -14.906     30.797  -0.484
stateAZ                 -1.939     25.818  -0.075
stateCA                 97.230     24.071   4.039
stateCO                 20.098     31.613   0.636
stateCT                 84.

In [25]:
predictions = predict(ltv.model, newdata = ltv, allow.new.levels = TRUE)

In [27]:
head(predictions)

1        2        3        4        5        6 
473.6598 376.3153 472.4803 719.8833 344.9284 576.5843